In [1]:
class Card():
    '''
    DOCSTRING: Class defining the individual cards
    '''
    
    def __init__(self, suit='', pip=''):
        self.pip = pip
        self.suit = suit
    
    def pips():
        pips = {'A':11,
                '2':2,
                '3':3,
                '4':4,
                '5':5,
                '6':6,
                '7':7,
                '8':8,
                '9':9,
                '10':10,
                'J':10,
                'Q':10,
                'K':10
               }
        return pips
    
    def suits():
        return ['♠', '♦', '♥', '♣']
    
    def pretty_card(self, card_tuple, show_firstcard):
        self.suit = card_tuple[0]
        self.pip = card_tuple[1]
        
        hidden_card_disp = [
        '┌───────┐',
        '│░░░░░░░│',
        '│░░░░░░░│',
        '│░░░░░░░│',
        '│░░░░░░░│',
        '│░░░░░░░│',
        '└───────┘'
        ]
                
        card_disp = [
            '┌───────┐',
            '│{a:<2}     │'.format(a=str(self.pip)),
            '│   {b:<2}  │'.format(b=''),
            '│   {c:<2}  │'.format(c=str(self.suit)),
            '│   {d:<2}  │'.format(d=''),
            '│     {e:>2}│'.format(e=str(self.pip)),
            '└───────┘'
                    ]
        
        if show_firstcard:
            return card_disp
        else:
            return hidden_card_disp

In [2]:
class Deck(Card):
    '''
    DOCSTRING: Class defining a deck of cards
    '''
    def __init__(self):
        Card.__init__(self)
        self.cards = []
        
    def shuffle(self):
        from random import shuffle
        self.cards = []
        pips = Card.pips()
        suits = Card.suits()
        
        for x, suit in enumerate(suits):
            for y, pip in enumerate(pips.keys()):
                self.cards.append((suit,pip))
        shuffle(self.cards)
        return self.cards
    
    def deal(self):
        dealt_card = self.cards[0]
        self.cards.pop(0)
        return dealt_card

In [3]:
class Hand(Card):
    '''
    DOCSTRING: Defines the hands of the player and the dealer
    INPUTS:
        Cards: A list of (suit, pip) tuples
        Value: Numeric value of the hand
    '''
    def __init__(self, cards=[]):
        Card.__init__(self)
        self.cards = cards
        self.value = 0
        
    def total(self):
        #Sum the value of the pips in the hand
        pip_value = []
        pips = Card.pips()
        for suit,pip in self.cards:
            pip_value.append(pips[pip])
        self.value = sum(pip_value)
        
        #Check whether the hand is busted
        if self.value > 21:
            for i in range(len(pip_value)):
                if pip_value[i] == 11:
                    pip_value[i] = 1
                    if sum(pip_value) <= 21:
                        break
        self.value = sum(pip_value)
        return self.value

    def add_card(self, new_card):
#         (suit, pip) = new_card
        self.cards.append(new_card)
        return self.cards
    
    def new_hand(self):
        self.cards = []
    
    def __str__(self):
        string_version = ', '.join(f'({suit},{pip})' for (suit,pip) in self.cards)
        return string_version

In [4]:
class Player(Hand):
    '''
    DOCSTRING: Defines the player
    INPUTS:
            HAND: The list of cards in the players hand (given as (suit, pip) tuples)
            MONEY: The amount of money the player has
            BET: The current bet for this round
    ATTRIBUTES: Hand, money, bet
    '''
    def __init__(self, hand=[], money = 0, bet_amt = 0):
        Hand.__init__(self)
        self.hand = hand
        self.money = money
        self.bet_amt = bet_amt
    
    def bet(self):
        while True:
            try:
                bet_amt = int(input("How much would you like to bet? (Current bankroll: ${v})\n".format(v = self.money)))
            except:
                print("You must enter a numeric value")
                continue
            else:
                if bet_amt > self.money:
                    print(f"You can't bet more money than you have! (You have ${self.money})")
                    continue
                elif bet_amt <= 0 or bet_amt%5!=0:
                    print('You must place a valid bet\n(Must be greater than 0 and divisble by 5).')
                    continue
                else:
                    self.bet_amt = bet_amt
                    break
    
    def bankroll(self):
        while True:
            try:
                money_amt = int(input("How much money do you have available to play?\n"))
            except:
                print("You must enter a numeric value")
                continue
            else:
                if money_amt <= 0 or money_amt%5!=0:
                    print('You must place a valid bet\n(Must be greater than 0 and divisible by 5).\n')
                    continue
                else:
                    self.money = money_amt
                    break
    
    def __str__(self):
        str_val = f"Player has a bankroll of ${self.money} and their current bet is ${self.bet_amt}."
        str_val +='\n'
        str_val += "Player's hand is: " + str(self.hand)
        return str_val
    

In [5]:
class Dealer(Hand):
    def __init__(self):
        Hand.__init__(self)
        self.hand = []
        self.money = 0
        self.bet_amt = 0
        
    def disp_hand(self):
        dealer_hand = self.hand
        string_version = '[(Hidden), '
        string_version += ', '.join(f'({suit},{pip})' for (suit,pip) in self.cards[1:])
        string_version += ']'
        return string_version
    
    def __str__(self):
        disp_hand = self.hand
        disp_hand[0] = 'Hidden'
        str_val = "Dealer's hand is: " + str(disp_hand)
        return str_val

In [6]:
def ascii_cards(cards, show_card):
    pretty_cards=[]
    for card in range(len(cards)):
        if card > 0:
            show_card = True
        new_card = Hand(cards[card]).pretty_card(cards[card],show_card)
        if pretty_cards == []:
            pretty_cards = new_card
        else:
            for line in range(len(pretty_cards)):
                pretty_cards[line]+=new_card[line]

    my_str = ''
    for line in range(len(pretty_cards)):
        for card in range(len(pretty_cards[0])):
            my_str += ''.join(str(pretty_cards[line][card]))
        my_str += '\n'

    return(my_str)

In [7]:
def print_table(dealer, players, show_firstcard):
    print("=== Dealer's Hand ===\n")
    print(ascii_cards(dealer.cards, show_firstcard)) 
    print("=== Player's Hand ===\n")
    print(ascii_cards(players.hand, True))

In [8]:
def check_winner(new_player, the_dealer, bj_mult):
    if new_player.total() == 21 and len(new_player.cards)==2:
        if the_dealer.total() == 21 and len(the_dealer.cards)==2:
            print('This round is a tie.')
        else:
            new_player.money += (new_player.bet_amt * bj_mult)
            print(f'Blackjack! ${new_player.bet_amt * bj_mult} added to your bankroll. New bankroll is ${new_player.money}')

    elif new_player.total() > 21:
#         print("You've lost (Hand value is {v}).".format(v = new_player.total()))
        new_player.money -= new_player.bet_amt
        print(f"You've lost (Hand value is {new_player.total()}). ")
        print(f'${new_player.bet_amt} subtracted from your bankroll. New bankroll is ${new_player.money}')

    elif the_dealer.total() > 21:
        new_player.money += new_player.bet_amt
        print(f"You win this round (Dealer hand value is {the_dealer.total()})!")
        print(f'${new_player.bet_amt} added to your bankroll. New bankroll is ${new_player.money}')

    elif new_player.total() < the_dealer.total():
        new_player.money -= new_player.bet_amt
        print(f"You've lost (Hand value is {new_player.total()}).")
        print(f'${new_player.bet_amt} subtracted from your bankroll. New bankroll is ${new_player.money}')

    elif new_player.total() > the_dealer.total():
        new_player.money += new_player.bet_amt
        print(f"You win this round (Dealer hand value is {the_dealer.total()})!")
        print(f'${new_player.bet_amt} added to your bankroll. New bankroll is ${new_player.money}')

    else:
        print('This round is a tie.')

In [9]:
def play_blackjack():
    # initialize globals
    from IPython.display import clear_output
    import time
    
    keep_playing = input('Welcome to virtual blackjack! Would you like to play now? (Yes or no)\n')
    the_deck = Deck()
    the_dealer = Dealer()
    players = []
    winner = ''
    new_player = Player()
    if keep_playing.lower() == 'yes':
        new_player.bankroll()
    
    while keep_playing.lower() == 'yes':
        new_player.bet()
        clear_output()
        bj_mult = 1.0
        
        #Deal cards
        if len(the_deck.cards) < 26:
            print('Shuffling the deck.')
            the_deck.shuffle()
        
        the_dealer.new_hand()
        new_player.new_hand()
        
        for i in range(0,4):
            if i%2==0:
                the_dealer.hand = the_dealer.add_card(the_deck.deal())
            else:
                new_player.hand = new_player.add_card(the_deck.deal())
        
        #Check for blackjack
        if new_player.total() == 21:
            if the_dealer.total() == 21:
                print('Both you and the dealer hit blackjack! This round is a tie.')
            else:
                print('Blackjack!\n')
                bj_mult = 1.5
        else:
            #Check whether the player has busted
            while new_player.total() <= 21:
                #Ask player whether they want to hit or stay
                print_table(the_dealer, new_player, False)
#                 time.sleep(.1)
                hit_or_stay = input("Your hand value is {v}. Hit or stay?\n".format(v = new_player.total()))
                if hit_or_stay.lower() == 'hit':
                    new_player.hand = new_player.add_card(the_deck.deal())
#                     clear_output(wait=True)
                    continue
                else:
                    break
#                 clear_output(wait=True)
            else:
                print("You've busted. (Hand value is {v}).".format(v = new_player.total()))

            #If player didn't bust, dealer then hits until they beat the player, bust, or total between 17 and 20
            if new_player.total() <= 21:
                dealer_stays = (
                    (the_dealer.total() >= 17) or 
                    (the_dealer.total() > new_player.total()) or 
                    (the_dealer.total()>=21)
                )
                while not(dealer_stays):
                    clear_output(wait=True)
                    print_table(the_dealer, new_player, True)
                    the_dealer.hand = the_dealer.add_card(the_deck.deal())
#                     time.sleep(.1)
                    proceed = input('Dealer hits. Their new hand value is {v}.\nPress enter to proceed.'.format(v = the_dealer.total()))
                    dealer_stays = (the_dealer.total() > 17) or (the_dealer.total() > new_player.total())
                    continue
        
        #Resolve the winnings, ask if player will play again
        clear_output(wait=True)
        print_table(the_dealer, new_player, True)
        check_winner(new_player, the_dealer, bj_mult)
#         time.sleep(.1)
        if new_player.money == 0:
            print("You're out of money! Bummer. Better luck next time.")
            break
        else:
            keep_playing = input('Keep playing? (Yes or no)\n')
            
        
    else:
        print('\n\nMaybe another time!')

In [10]:
play_blackjack()

=== Dealer's Hand ===

┌───────┐┌───────┐
│9      ││7      │
│       ││       │
│   ♦   ││   ♠   │
│       ││       │
│      9││      7│
└───────┘└───────┘

=== Player's Hand ===

┌───────┐┌───────┐┌───────┐┌───────┐
│J      ││4      ││A      ││10     │
│       ││       ││       ││       │
│   ♦   ││   ♥   ││   ♠   ││   ♥   │
│       ││       ││       ││       │
│      J││      4││      A││     10│
└───────┘└───────┘└───────┘└───────┘

You've lost (Hand value is 25). 
$250 subtracted from your bankroll. New bankroll is $250
Keep playing? (Yes or no)
yyes


Maybe another time!
